In [ ]:
%%writefile lattice_3x3_springs_play.py
from manim import *
import numpy as np

class Lattice3x3Springs(Scene):
    def construct(self):
        N = 3
        a = 1.2
        k = 35.0
        c = 0.4
        m = 1.0
        dt = 1/120

        coils = 8
        amp = 0.12
        end_cap_frac = 0.12

        steps_per_frame = 1  # raise for faster/more obvious motion

        def idx(i, j): return i * N + j
        fixed = {idx(0, 0), idx(0, N - 1)}

        x0 = np.zeros((N*N, 2), dtype=float)
        for i in range(N):
            for j in range(N):
                x0[idx(i, j)] = [j*a, (N-1-i)*a]
        x0 -= np.mean(x0, axis=0)

        springs = []
        for i in range(N):
            for j in range(N):
                if j + 1 < N: springs.append((idx(i, j), idx(i, j+1), a))
                if i + 1 < N: springs.append((idx(i, j), idx(i+1, j), a))

        pos = x0.copy()
        vel = np.zeros_like(pos)
        pos[idx(1, 1)] += [0.6, -0.45]  # big poke so it’s obvious

        def to3(p2): return np.array([p2[0], p2[1], 0.0])

        def spring_pts(p, q):
            d = q - p
            L = np.linalg.norm(d)
            if L < 1e-6:
                return [to3(p), to3(q)]
            u = d / L
            perp = np.array([-u[1], u[0]])
            cap = end_cap_frac * L
            core = max(L - 2*cap, 1e-6)

            pts = [p, p + u*cap]
            for n in range(coils*2):
                t = (n + 1) / (coils*2 + 1)
                base = p + u*(cap + t*core)
                sign = 1 if n % 2 == 0 else -1
                pts.append(base + sign*amp*perp)
            pts += [q - u*cap, q]
            return [to3(x) for x in pts]

        def step():
            nonlocal pos, vel
            F = -c * vel
            for i, j, L0 in springs:
                d = pos[j] - pos[i]
                L = np.linalg.norm(d)
                if L > 1e-9:
                    e = d / L
                    f = k * (L - L0) * e
                    F[i] += f
                    F[j] -= f
            acc = F / m
            for fidx in fixed:
                pos[fidx] = x0[fidx]
                vel[fidx] = 0.0
                acc[fidx] = 0.0
            vel += acc * dt
            pos += vel * dt

        # visuals
        title = Text("3×3 Mass–Spring Lattice", font_size=34).to_edge(UP)
        self.add(title)

        masses = VGroup(*[
            Dot(radius=0.10, color=ORANGE if n in fixed else WHITE)
            for n in range(N*N)
        ])
        springs_vg = VGroup(*[
            VMobject().set_stroke(width=4)
            for _ in springs
        ])
        self.add(springs_vg, masses)

        lattice = VGroup(springs_vg, masses)

        def update_all(_):
            for _ in range(steps_per_frame):
                step()

            for n, dot in enumerate(masses):
                dot.move_to(to3(pos[n]))

            for s_mob, (i, j, _) in zip(springs_vg, springs):
                s_mob.set_points_as_corners(spring_pts(pos[i], pos[j]))

        self.play(UpdateFromFunc(lattice, update_all), run_time=6, rate_func=linear)



In [ ]:
!manim -ql --fps 15 --disable_caching lattice_3x3_springs_play.py Lattice3x3Springs

In [ ]:
!ls -lh media/videos/lattice_3x3_springs_play/*/Lattice3x3Springs.mp4

In [ ]:
from IPython.display import Video
Video("media/videos/lattice_3x3_springs_play/480p15/Lattice3x3Springs.mp4", embed=True)

In [ ]:
import numpy as np

def lattice_normal_modes(N=3, a=1.2, k=35.0, m=1.0, fixed_nodes=None, nmodes=10):
    """
    2D mass-spring lattice, nearest-neighbor springs, small oscillations about equilibrium.

    DOFs: for each node n, displacement (ux, uy).
    K assembled from springs: k * (e e^T) contributions along spring direction e.
    Mass matrix M = m I (all equal masses).

    fixed_nodes: iterable of node indices (0..N*N-1) to pin (remove their DOFs).
    Returns: (omegas, modes_full, free_dof_indices)
      - omegas: sorted angular frequencies (rad/s)
      - modes_full: eigenvectors in full DOF space (size 2*N*N), with fixed DOFs = 0
      - free_dof_indices: list of free DOF indices used internally
    """
    nn = N * N

    def idx(i, j):  # node index
        return i * N + j

    # equilibrium positions (same as your animation: grid centered)
    x0 = np.zeros((nn, 2), dtype=float)
    for i in range(N):
        for j in range(N):
            x0[idx(i, j)] = [j * a, (N - 1 - i) * a]
    x0 -= np.mean(x0, axis=0)

    # spring list (nearest neighbors)
    springs = []
    for i in range(N):
        for j in range(N):
            if j + 1 < N:
                springs.append((idx(i, j), idx(i, j + 1)))
            if i + 1 < N:
                springs.append((idx(i, j), idx(i + 1, j)))

    # fixed nodes
    fixed_nodes = set(fixed_nodes or [])
    fixed_dofs = set()
    for n in fixed_nodes:
        fixed_dofs.add(2*n)     # x DOF
        fixed_dofs.add(2*n + 1) # y DOF

    ndof = 2 * nn
    K = np.zeros((ndof, ndof), dtype=float)

    # assemble stiffness matrix
    for i, j in springs:
        d = x0[j] - x0[i]
        L = np.linalg.norm(d)
        if L < 1e-12:
            continue
        e = d / L                        # 2-vector
        Ke = k * np.outer(e, e)          # 2x2

        # DOF indices for node i and j
        ii = [2*i, 2*i+1]
        jj = [2*j, 2*j+1]

        # Add contributions:
        # K_ii += Ke, K_jj += Ke, K_ij -= Ke, K_ji -= Ke
        K[np.ix_(ii, ii)] += Ke
        K[np.ix_(jj, jj)] += Ke
        K[np.ix_(ii, jj)] -= Ke
        K[np.ix_(jj, ii)] -= Ke

    # build list of free DOFs
    free = [d for d in range(ndof) if d not in fixed_dofs]
    Kf = K[np.ix_(free, free)]

    # generalized eigenproblem K u = ω^2 M u, with M = m I  =>  (K/m) u = ω^2 u
    evals, evecs = np.linalg.eigh(Kf / m)

    # numerical cleanup: negative tiny eigenvalues -> 0
    evals = np.clip(evals, 0.0, None)
    omegas = np.sqrt(evals)

    # sort
    order = np.argsort(omegas)
    omegas = omegas[order]
    evecs = evecs[:, order]

    # expand modes back to full DOF (fixed DOFs = 0)
    modes_full = np.zeros((ndof, evecs.shape[1]), dtype=float)
    for col in range(evecs.shape[1]):
        modes_full[free, col] = evecs[:, col]

    # print a table
    print(f"N={N}, springs={len(springs)}, free DOFs={len(free)}")
    print("mode   omega (rad/s)     f (Hz)")
    for r in range(min(nmodes, len(omegas))):
        f_hz = omegas[r] / (2*np.pi)
        print(f"{r:4d}   {omegas[r]:12.6g}   {f_hz:10.6g}")

    return omegas, modes_full, free

# Example: match your earlier “pin top corners”
def top_corners_fixed(N):
    def idx(i, j): return i * N + j
    return [idx(0, 0), idx(0, N-1)]

# --- Run it ---
omegas, modes_full, free = lattice_normal_modes(
    N=3, a=1.2, k=35.0, m=1.0,
    fixed_nodes=top_corners_fixed(3),
    nmodes=12
)


In [ ]:
!git status
!git add working_animation.ipynb
!git commit -m "Describe what I changed"
!git push